In [1]:
import os
os.path.join("CSV_DataBase","User.csv")

'CSV_DataBase/User.csv'

In [2]:
import pandas as pd
pd.read_csv('CSV_DataBase/Record/222.csv').columns

Index(['Unnamed: 0', 'Date', 'ID', 'Name', 'Number Sold', 'Discount', 'Price',
       'Event', 'Weather', 'Restaurant Type', 'Restaurant ID',
       'Restaurant Address', 'Restaurant Annual Sale'],
      dtype='object')

In [3]:
import pandas as pd
from py2neo import Graph, Node, Relationship, RelationshipMatcher
import streamlit as st
import os
from Model import Model
import joblib
class Central:
    def __init__(self): 
        self.folder = "CSV_DataBase"
        self.record = "Record"
        self.files = {"Manager":"Manager.csv","Owner":"Owner.csv","Store":"Store.csv","Sales":"Record.csv"}
        self.Model = Model()
    def exist(self,file):
        return os.path.exists(file)
    def signup(self, info):

        tar = os.path.join(self.folder,self.files["Owner"])
        if exit(tar):
            df = pd.read_csv(tar)
            df.loc[len(df)] = info
        else:
            df = pd.DataFrame([info])
        df.to_csv(tar,index=False)

    def signup_manager(self, owner,info):
        tar = os.path.join(self.folder,self.files["Manager"])
        info["owner"] = owner
        if info['data'] and info['data_type']:
            if info['data_type'] == 'File':
                print(info['data'])
                data_for_enhanced = pd.read_csv(info['data'])
                cmodel = joblib.load(st.secrets['model']['model_path'])
                model,path = self.Model.train_model_enhanced(data_for_enhanced,cmodel,info['account'])
                info['model'] = path
        
        if exit(tar):
                df = pd.read_csv(tar)
                df.loc[len(df)] = info
        else:
                df = pd.DataFrame([info])
        return True

    def remove_subaccount(self,account,selected):
        db = self.get_db() 
        start_node = db.nodes.match("User", account=account).first()
        end_node = db.nodes.match("User", account=selected).first()
        relationship = db.match(nodes=(start_node, end_node), r_type="owns").first()  # 替换为实际的关系类型
        #print(relationship)
        if not relationship:
            db.separate(relationship)
            #print("released")
            db.push(relationship)
            return True
        return False
    def add_subaccount(self,account,selected):
        db = self.get_db() 
        start_node = db.nodes.match("User", account=account).first()
        end_node = db.nodes.match("User", account=selected).first()
        if start_node and end_node and end_node.get('role') == 'Manager':
            db.create(Relationship(start_node,'owns',end_node))
            return True
        return False

    def change_password(self,account,password):
        db = self.get_db()
        user_node = db.nodes.match("User", account=account).first()
        if not user_node:
            return False
        user_node['password'] = password
        db.push(user_node)
        return True
    def api_verification(self, api, account):
        db = self.get_db()
        api_node = db.nodes.match("API", key=api).first()
        if not api_node:
            return False, False

        if api_node['number'] >= 1:
            return True, False
        
        else:
            return True,True
        

    def account_verification(self, account, password):
        db = self.get_db()
        user_node = db.nodes.match("User", account=account).first()
        if not user_node:
            return False, False
        return (user_node['account'], user_node['role']) if user_node['password'] == password else (True, False)
    
    def find_subaccount(self,account):
        db =self.get_db()
        user_node = db.nodes.match("User", account=account).first()
        if not user_node:
            return []
        rela = RelationshipMatcher(db).match(nodes=[user_node],r_type="owns",)
        
        return [r.end_node for r in rela]
    
    def get_restaurant_from_owner(self,owneraccount):
        db = self.get_db()
        node = db.nodes.match('User',account=owneraccount).first()
        
        subaccounts = db.match((node,),r_type='owns').all()
        dfs = []
        for subaccount in subaccounts:
            restaurant = db.match((subaccount.end_node,),r_type='is').first().end_node
            dfs.append(self.restore_df_from_neo4j(restaurant))
        return pd.concat(dfs,ignore_index=True)




    def check_pro(self,account):
        db = self.get_db()
        user_node = db.nodes.match("User", account=account).first()
        if not user_node:
            return False
        if user_node.get('Pro'):
            return True
        return False
    
    def write_style(self,text,icon = '',iconsize = 30,ALLSIZE = 30):
        
        text = """
                            <div style="display: flex; align-items: center; justify-content: left; font-size: ALLSIZEpx;">
                                <!-- 正常显示 -->
                                <span style="font-size: ICONSIZE px;">ICON</span>
                                <!-- 渐变字体 Pro -->
                                <span style="font-weight: bold; 
                                            background: -webkit-linear-gradient(left, red, orange);
                                            background: linear-gradient(to right, red, orange);
                                            -webkit-background-clip: text;
                                            -webkit-text-fill-color: transparent;
                                            margin-left: 5px;">
                                    TEXT
                                </span>
                            </div>
                            """
        return text.replace("TEXT",text).replace("ICON",icon).replace("ICONSIZE",iconsize).replace("ALLSIZE",ALLSIZE)
    def put(self,text,font,tar = st,color='white',height = 2):
        tar.markdown(f"""
        <div style='text-align: left; color: {color}; font-size: {font}px; font-family: Arial; line-height: {height};'>
            {text}
        </div>
        """, unsafe_allow_html=True)




In [66]:

class Central:
    def __init__(self): 
        self.folder = "CSV_DataBase"
        self.record = "Record"
        self.files = {"API":"API.csv","Manager":"Manager.csv","Owner":"Owner.csv","Store":"Store.csv","Sales":"Record.csv"}
        self.Model = Model()
    def exist(self,file):
        return os.path.exists(file)
    def signup(self, info):

        tar = os.path.join(self.folder,self.files["Owner"])
        if exit(tar):
            df = pd.read_csv(tar)
            df.loc[len(df)] = info
        else:
            df = pd.DataFrame([info])
        df.to_csv(tar,index=False)

    def signup_manager(self, owner,info):
        tar = os.path.join(self.folder,self.files["Manager"])
        info["owner"] = owner
        if info['data'] and info['data_type']:
            if info['data_type'] == 'File':
                data_for_enhanced = pd.read_excel(info['data'])
                print('{}'.format(info['account'])+'.csv')
                data_add = os.path.join(self.folder,self.record,'{}'.format(info['account'])+'.csv')
                print(data_add)
                data_stored = data_for_enhanced.to_csv(data_add)
                info['data_add'] = data_add
                cmodel = joblib.load(st.secrets['model']['model_path'])
                model,path = self.Model.train_model_enhanced(data_for_enhanced,cmodel,info['account'])
                info['model'] = path
        
        if exit(tar):
                df = pd.read_csv(tar)
                df.loc[len(df)] = info
        else:
                df = pd.DataFrame([info])
        
        df.to_csv(tar,index=False)
        return True


    def remove_subaccount(self,account,selected):

        owner = account
        manager = selected
        tar = os.path.join(self.folder,self.files["Manager"])
        print(tar)
        if self.exist(tar):
            df = pd.read_csv(tar)
            if not df[(df["account"]==manager) & (df["owner"]==owner)].empty:
                df.loc[(df["account"] == manager) & (df["owner"] == owner), 'owner'] = None
                df.to_csv(tar,index=False)
        return False
    
    def add_subaccount(self,account,selected):
        owner = account
        manager = selected
        tar = os.path.join(self.folder,self.files["Manager"])
        if self.exist(tar):
            df = pd.read_csv(tar)
            if not df[df["account"]==manager].empty:
                df.loc[df["account"]==manager,'owner'] = owner
                df.to_csv(tar,index=False)


    def change_password(self,account,password):
        tar = os.path.join(self.folder,self.files["Owner"])
        tar_m = os.path.join(self.folder,self.files["Manager"])
        if self.exist(tar):
            df = pd.read_csv(tar)
            if not df[df["account"]==account].empty:
                df.loc[df["account"]==account,'password'] = password
                df.to_csv(tar,index=False)
        elif self.exist(tar_m):
            df = pd.read_csv(tar_m)
            if not df[df["account"]==account].empty:
                df.loc[df["account"]==account,'password'] = password
                df.to_csv(tar_m,index=False)

    def api_verification(self, api, account):
        tar = os.path.join(self.folder, self.files["API"])
        if self.exist(tar):
            df = pd.read_csv(tar)
            print(df[df["key"] == api].empty)
            if not df[df["key"] == api].empty:
                if (df[df['key'] == api]['connection'] >= 1).any():
                    return True, False
                else:
                    return True,True
            else:
                return False,False
        else: pd.DataFrame([{"key":"MSBA",'connection':0}]).to_csv(tar,index=False)
        
    def account_verification(self, account, password):
        tar = os.path.join(self.folder, self.files["Owner"])
        tar_m = os.path.join(self.folder, self.files["Manager"])

        if self.exist(tar):
            df = pd.read_csv(tar)
            if not df[df['account']==account].empty:
                acc_e = True
                role = "Owner"
                if not df[df['account'] == account].empty and df[df['account'] == account]['password'].iloc[0] == password:
                    pass_e = True
                else:
                    pass_e = False
            else: pass_e = False
            if acc_e:
                if pass_e:
                    return account, role
                else:
                    return acc_e,pass_e
        elif self.exist(tar_m):
            df = pd.read_csv(tar_m)
            if not df[df['account']==account].empty:
                acc_e = True
                role = 'Manager'
                if not df[df['account'] == account].empty and df[df['account'] == account]['password'].iloc[0] == password:
                    pass_e = True
                else:
                    pass_e = False
            else: pass_e = False
            if acc_e:
                if pass_e:
                    return account, role
                else:
                    return acc_e,pass_e
        return False,False
    def find_subaccount(self,account):
        tar = os.path.join(self.folder, self.files["Manager"])
        if self.exist(tar):
            df = pd.read_csv(tar)
            if not df[df['owner']==account].empty:
                return df[df['owner']==account]['account'].values.tolist()
            else: return []
        return []
    
    def get_restaurant_from_owner(self,owneraccount):
        subaccounts = self.find_subaccount(owneraccount)
        if subaccounts:
            tar = os.path.join(self.folder, self.files["Manager"])
            df = pd.read_csv(tar)
            dfs = []
            for subaccount in subaccounts:
                dft = pd.read_csv(df[df['account'] == subaccount]["data_add"].iloc[0])
                dfs.append(dft)

        return pd.concat(dfs,ignore_index=True)

    def check_pro(self,account):
        tar = os.path.join(self.folder, self.files["Owner"])
        tarm = os.path.join(self.folder, self.files["Manager"])
                
        if self.exist(tar):
            df = pd.read_csv(tar)
            if not df[df['account']==account].empty:
                filtered_df = df[df['account'] == account]
                if not filtered_df.empty and filtered_df['Pro'].iloc[0] != False:
                    return True

        elif self.exist(tarm):
            df = pd.read_csv(tarm)
            if not df[df['account']==account].empty:
                filtered_df = df[df['account'] == account]
                if not filtered_df.empty and filtered_df['Pro'].iloc[0] != False:
                    return True

        return False

cen = Central()
user_info = {"account":111,'password':111,'email':111,'role':'Owner',"Pro":"MSBA"}
cen.signup(user_info)
cen.signup_manager(111,{"account":222,'password':212,'email':212,'role':"Manager","Pro":False,'address':122,'data':"test_data/R121_signup.xlsx","data_type":"File"})
cen.add_subaccount(111,222)
cen.change_password(111,2323)
cen.api_verification("MSBA",111)
cen.account_verification(111,2323)
cen.find_subaccount(111)
cen.get_restaurant_from_owner(111)
cen.check_pro(111)

222.csv
CSV_DataBase/Record/222.csv
New User Model saved at: Model/222_model_rf.pkl
False


True

In [32]:
os.path.join("CSV_DataBase","Record",'file.txt')

'CSV_DataBase/Record/file.txt'

(np.float64(526810.0), 0.0)